In [1]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [2]:
a = './result/detectRS_0.538.csv'
b = './result/pvt_b3_14epoch.csv'
c = './result/swin_0.559.csv'
d = './result/pvt_seryung.csv'
e = './result/efficientD4.csv'
f = './result/univ101_24.csv'
g = './result/swinSGD.csv'
h = './result/detecto_resnet.csv'
i = './result/dcn_15epo_highScoreBox_adam.csv'
j = './result/dcn_12epo_highsmallBox_adam.csv'

A = './submit/pvtB0,B3_univ101_effD4_swinSGD,Adam_detRSnet,next_wbf_0.5_0.0001.csv'
B = './submit/dcnHScore_pvtB0,B3_univ101_effD4_swinSGD,Adam_detRSnet,next_wbf_all.csv'
submission_files = [a, b, c,d] # submission lists

submission_df = list(pd.read_csv(file) for file in submission_files)

In [3]:
image_ids = submission_df[0]['image_id'].tolist()

In [4]:
annotation = '../../../dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [5]:
prediction_strings = []
file_names = []
iou_thr = 0.6

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    iou_thr = 0.5
    skip_box_thr = 0.01
    sigma = 0.1
    
    if len(boxes_list):
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')


In [6]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission_ensemble.csv')

submission.head()

,PredictionString,image_id
0,7 0.9889448881149292 604.962646484375 515.0734...,test/0000.jpg
1,5 0.7057606 135.368408203125 0.453578114509582...,test/0001.jpg
2,1 0.8487153053283691 776.3338623046875 407.565...,test/0002.jpg
3,9 0.5672592 101.13658142089844 194.02061462402...,test/0003.jpg
4,1 0.6174700856208801 200.9127960205078 287.216...,test/0004.jpg
